In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


seasons = 2
Game = Regency(True)
clear_output()
Game.add_relationship('Go', 'Fh', At_War=1)
for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.Seasons[0][1]['Override']['Go'] = (22, 'enemy', 'Fh')
    Game.take_domain_actions()
    Game.war_move()
    Game.clean_up()
    Game.clear_screen()

df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
print(set(df['Action']))
print()
print(set(df[df['Success?']==True]['Action']))
Game.War


{'disband_mercenaries', 'declare_war', 'diplomacy_force_tribute', 'build_ship', 'diplomacy_respond_to_brigandage', 'espionage_intrigue', 'investure_invest_friend', 'realm_magic_demagogue_enemy', 'espionage_assassination', 'disband_army', 'muster_mercenaries', 'espionage_discover_troop_movements', 'diplomacy_trade_agreement', 'espionage_heresy', 'espionage_diplomatic_details', 'move_troops_into_enemy_terrirtory', 'realm_magic_alchemy', 'realm_magic_bless_land', 'espionage_corruption', 'rule_holdings', 'contest_provence', 'investiture_become_vassal_friend', 'diplomacy_form_alliance', 'adventuring', 'contest_holding', 'realm_magic_demagogue_friend', 'decree_general', 'diplomacy_troop_permission', 'grant', 'muster_levies', 'agitiate_for_friend', 'decree_asset_seizure', 'realm_magic_blight', 'realm_magic_death_plague', 'investure_divest_enemy', 'realm_magic_mass_destruction', 'muster_army', 'Lieutenant', 'create_holding'}

{'disband_mercenaries', 'declare_war', 'diplomacy_force_tribute', 'b

,Year,Location,Event,Notes
0,"Winter, year 1524 HC",Awallaigh,Occupation of Awallaigh,The Gorgon occupied Awallaigh with 1 Dwarf Cro...
0,"Spring, year 1524 HC",Awallaigh,Occupation of Awallaigh,The Gorgon occupied Awallaigh with 1 Dwarf Cro...


In [8]:
Game.War.drop_duplicates().iloc[1]['Notes']

'The Gorgon occupied Awallaigh with 1 Dwarf Crossbows, 2 Dwarf Guards, 1 Goblin Cavalry, 2 Goblin Infantry quashing all rival law holdings.'

In [3]:
df[df['Action'].str.contains('move_troops')]

,Regent,Actor,Action Type,Action,Decision,Target Regent,Provence,Target Provence,Target Holding,Success?,Base Reward,State,Invalid,Message
59,Go,The Gorgon,Action,move_troops_into_enemy_terrirtory,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",,Awallaigh,,,True,15,"[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",False,"The Gorgon moved Dwarf Guards, Goblin Infantry..."


In [4]:
Game.Troops[Game.Troops['Provence']=='Awallaigh']

,Regent,Provence,Type,Cost,CR,Garrisoned,Home,Injury
76,Go,Awallaigh,Dwarf Guards,2,2.0,0,,0
77,Go,Awallaigh,Goblin Infantry,1,1.0,0,,0
78,Go,Awallaigh,Goblin Infantry,1,1.0,0,,0
79,Go,Awallaigh,Goblin Cavalry,2,1.0,0,,0
80,Go,Awallaigh,Dwarf Guards,2,2.0,0,,0
81,Go,Awallaigh,Dwarf Crossbows,2,2.0,0,,0


In [10]:
Game.Relationships[Game.Relationships['Regent']=='Fh']

,Regent,Other,Diplomacy,Payment,Vassalage,At War,Trade Permission
80,Fh,FD,2,0,0,0,0
81,Fh,Go,-4,0,0,0,0
82,Fh,Is,3,0,6,0,0


In [6]:
Game.Regents[Game.Regents['Regent']=='LPA']

,Regent,Full Name,Bloodline,Culture,Player,Class,Level,Alignment,Race,Str,...,Insight,Deception,Persuasion,Regency Points,Gold Bars,Regency Bonus,Attitude,Alive,Divine,Arcane
1,LPA,Life and Protection of Avanalae (Medhlorie Hae...,An,A,False,Priest,5,NN,Human,0,...,1,1,3,37.0,17.0,3,Normal,True,True,False


In [9]:
df[df['Regent']=='Fh']

,Regent,Actor,Action Type,Action,Decision,Target Regent,Provence,Target Provence,Target Holding,Success?,Base Reward,State,Invalid,Message
41,Fh,Llytha Damaan,Bonus,agitiate_for_friend,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",Is,,"Awallaigh, Bhindraith, Braethindyr, Cwmbheir, ...",,False,0,"[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...",False,Fh failed to change peoples minds about Is in ...
42,Fh,Fhieleraenne (Tuarhievel),Bonus,agitiate_for_friend,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",Is,,"Awallaigh, Bhindraith, Braethindyr, Cwmbheir, ...",,True,3,"[1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...",False,Fh used their influence to rally the people be...
43,Fh,Fhieleraenne (Tuarhievel),Action,realm_magic_alchemy,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",,,,,True,8,"[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...",False,"Fhieleraenne (Tuarhievel) cast 'Alchemy', gene..."
25,Fh,Fhieleraenne (Tuarhievel),Action,investure_divest_enemy,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Is,,,,False,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",False,Fhieleraenne (Tuarhievel) Failed to divest The...
32,Fh,Fhieleraenne (Tuarhievel),Action,agitiate_for_friend,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Is,,"Awallaigh, Bhindraith, Braethindyr",,True,0,"[0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...",False,Fh agitated the people against GG in the follo...
28,Fh,Fhieleraenne (Tuarhievel),Action,realm_magic_alchemy,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",,,,,True,8,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",False,"Fhieleraenne (Tuarhievel) cast 'Alchemy', gene..."
41,Fh,Fhieleraenne (Tuarhievel),Action,investiture_become_vassal_friend,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Is,,,,True,5,"[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...",False,Fhieleraenne (Tuarhievel) became Emerald Queen...
25,Fh,Fhieleraenne (Tuarhievel),Action,diplomacy_respond_to_brigandage,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",,,,,False,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",False,82 Fhieleraenne (Tuarhievel)\nName: Full Na...
